# Lab 3 - The Matrix Form and The Duality Theory

<b>Information on group members:</b><br>
1) 156073, Maciej Janicki <br>
2) 156049, Jakub Kubiak

In [35]:
from pulp import *  
import numpy as np
import pandas as pd
import itertools
from IPython.display import display

# 1) The Matrix Form - Fundamental Insight (finish this part to get 3.0)

1.1) Given is the below (primal) linear problem:

primal problem:<br>
MAXIMIZE<br>
4*x1 + 2*x2 + 3*x3 <br>

SUBJECT TO<br>
1_constraint: x1 + x2 <= 10<br>
2_constraint: 2*x2 + x3 <= 12<br>
3_constraint: 3*x1 + 2*x3 <= 15<br>
4_constraint: x1 + x2 + x3 <= 20<br>

VARIABLES<br>
x1 Continuous<br>
x2 Continuous<br>
x3 Continuous<br>

1.2) Use the PuLP library to solve the above problem. Identify the optimal solution: the values for basic variables and the corresponding value for the objective function.

In [36]:
### Model here:
model = LpProblem("Problem1", sense=LpMaximize)
x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)
x3 = LpVariable(name="x3", lowBound=0)
z = 4 * x1 + 2 *x2 + 3*x3
constraint1 = x1 + x2 <=10
constraint2 = 2*x2 + x3 <=12
constraint3 = 3*x1 + 2*x3 <=15
constraint4 = x1 + x2 + x3 <=20
model += constraint1
model += constraint2
model += constraint3
model += constraint4
model += z 
model



Problem1:
MAXIMIZE
4*x1 + 2*x2 + 3*x3 + 0
SUBJECT TO
_C1: x1 + x2 <= 10

_C2: 2 x2 + x3 <= 12

_C3: 3 x1 + 2 x3 <= 15

_C4: x1 + x2 + x3 <= 20

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous

In [37]:
solution = model.solve(PULP_CBC_CMD(msg=False))
print(f"Model found the solution: Z={model.objective.value():.2f},for x1={x1.value():.2f}, x2={x2.value():.2f} and x3={x3.value():.2f} the solution is {LpStatus[model.status]}")

Model found the solution: Z=31.43,for x1=4.43, x2=5.57 and x3=0.86 the solution is Optimal


1.3) In this exercise, you are asked to identify ALL basic (feasible and not) solutions to the above problem. We will do this naively. Specifically, you are asked to use the fundamental insight to build a final simplex tableau for each possible base. Therefore, you need first to initialize the data: c, b, A matrixes, and it is suggested to initialize the auxiliary matrix M defined as M = A + (concatenate) I (identity matrix). Note that the problem should be formulated in the augmented form. Then, you have to iterate over each possible base B, compute B-1, and other relevant parts for the simplex tableau.<br><br>
a) Identify the optimal solution using the optimality condition; print it (Z value and values for basic variables); compare thus derived solution with the optimum found using the PuLP library (obviously, both solutions should be the same). <br>
b) Count the number of feasible and infeasible solutions. How many (all) basic solutions to the problem can be identified? <br><br>
It is suggested to use the NumPy library for performing matrix operations. 

In [38]:
### Initialize the data here:
c = np.array([4, 2, 3])
b = np.array([10, 12, 15, 20])
A = np.array([[1,1,0],
     [0,2,1],
     [3,0,2],
     [1,1,1]])
M = np.hstack((A, np.identity(n=4)))


<b> Important note: the below is just a proposition. You can solve the problem in your own way. </b>

You can define an auxiliary method constructing a final simplex tableau for a given base.  Here, "base" is a list of columns (integers) for the base. Note that the functions in python can return multiple objects and you can use this functionality to return<br>
- the inversed base<br>
- coefficients in the row 0 for slack variables<br>
- right side values (except the objective function value)<br>
- the objective function value<br>
- the coefficients for decision variables in row 0 <br>
- the coefficients for decision variables in rows 1+<br>

Note that if BI cannot be built (it is possible), the method may return None in order to notify the executive method about this exception. 



In [39]:
def getFinalTableau(base,c, c_b, b, A, M):
    # TODO
    B = M[:,base]
    try:
        B_inv = np.linalg.inv(B)
    except np.linalg.LinAlgError:
        return None
    row0_c_slacks = c_b @ B_inv
    rs_values = B_inv@b
    row0_dec_vars = c_b@B_inv@A - c
    row1_dec_vars = B_inv@A
    return B_inv, row0_c_slacks, rs_values, row0_dec_vars, row1_dec_vars

In [40]:
from itertools import combinations
padded_c = np.hstack((c,[0,0,0,0]))
n_optimal_and_feasible = 0
n_feasible = 0
n = 0
n_non_invertable = 0
for base in combinations(range(7), r=4):
    n +=1
    c_b = padded_c[list(base)]
    ret = getFinalTableau(list(base),c,c_b,b,A,M)
    if ret is None:
        n_non_invertable +=1
        continue
    top_left = ret[3]
    bottom_left = ret[4]
    top_right = ret[1]
    bottom_right = ret[0]
    rs_bottom = ret[2]
    rs_top = c_b@ret[0]@b

    top = np.hstack((top_left,top_right,rs_top))
    bottom = np.hstack((bottom_left, bottom_right, rs_bottom.reshape(-1,1)))
    table = np.vstack((top,bottom))
    df = pd.DataFrame(table, columns=["x1","x2","x3","x4","x5","x6","x7","rs"])

    if np.all(rs_bottom >= 0):
        n_feasible +=1
        if np.all(np.hstack((top_left,top_right)) >= -0.00001):
            print(df.round(2))
            print(f"Optimal\tZ={rs_top:.2f}.")
            print("For:", end="")
            for i,x in enumerate(base):
                # if x not in [0, 1, 2]:
                #   break
                print(f"\tx{x}={rs_bottom[i]:.2f}")


print(f"For {n_non_invertable} bases matrix B was not invertible.")
print(f"There are {n_feasible} feasible solutions")
print(f"There are {n-n_feasible} infeasible solutions")
print(f"There are total of {n} solutions")

    x1   x2   x3    x4    x5    x6   x7     rs
0 -0.0  0.0  0.0  0.57  0.71  1.14  0.0  31.43
1  1.0  0.0  0.0  0.57 -0.29  0.14  0.0   4.43
2  0.0  1.0  0.0  0.43  0.29 -0.14  0.0   5.57
3 -0.0  0.0  1.0 -0.86  0.43  0.29 -0.0   0.86
4  0.0  0.0  0.0 -0.14 -0.43 -0.29  1.0   9.14
Optimal	Z=31.43.
For:	x0=4.43
	x1=5.57
	x2=0.86
	x6=9.14
For 4 bases matrix B was not invertible.
There are 8 feasible solutions
There are 27 infeasible solutions
There are total of 35 solutions


## 2) The Duality Theory (finish this part + part 1 + to get 5.0)

2.1) Model the dual problem to the above solved primal one, using the PuLP library. Then, solve it and compare the derived optimum with the optimum for the primal problem. Are they equal?

In [41]:
### Model here:
model2 = LpProblem("Problem2", sense=LpMinimize)
y1 = LpVariable(name="y1", lowBound=0)
y2 = LpVariable(name="y2", lowBound=0)
y3 = LpVariable(name="y3", lowBound=0)
y4 = LpVariable(name="y4", lowBound=0)

w = 10 * y1 + 12 * y2 + 15 * y3 + 20 * y4
model2 += y1 + 3 * y3 + y4 >= 4
model2 += y1 + 2 * y2  + y4 >= 2
model2 += y2 + 2*y3 + y4 >= 3
model2 +=w
model2

Problem2:
MINIMIZE
10*y1 + 12*y2 + 15*y3 + 20*y4 + 0
SUBJECT TO
_C1: y1 + 3 y3 + y4 >= 4

_C2: y1 + 2 y2 + y4 >= 2

_C3: y2 + 2 y3 + y4 >= 3

VARIABLES
y1 Continuous
y2 Continuous
y3 Continuous
y4 Continuous

In [42]:
solution2 = model2.solve(PULP_CBC_CMD(msg=False))
print(f"Model found the solution: W={model2.objective.value():.2f},for y1={y1.value():.2f}, y2={y2.value():.2f}, y3={y3.value():.2f} and y4={y4.value():.2f}. The solution is {LpStatus[model.status]}.")


Model found the solution: W=31.43,for y1=0.57, y2=0.71, y3=1.14 and y4=0.00. The solution is Optimal.


2.2) This exercise is based on the exercise 1.3 (copy & paste solution). Here, you are asked to iterate over all basic solutions (as in 1.3) and store them along with their complementary dual solutions. Solutions should be stored in the PRIMAL_DUAL_SOLUTIONS list and sorted according to the objective value Z = W. Analyze their optimality and feasibility. Finally, you have to display all basic solutions wlong with their complementary solutions (you can use the provided piece of code written using the pandas library). <br><br>

PRIMAL_DUAL_SOLUTIONS is defined as a table consisting of n rows, where n is the number of basic solutions to the problem, and 21 columns. The columns are defined as follows:<br>
Col. 1: The objective value Z<br>
Col. 2-4: The values for decision variables (primal solution)<br>
Col. 5-8: The values for slack variables (primal solution)<br>
Col. 9: P_F = Y or N, Y/N = primal solution is feasible/infeasible<br>
Col. 10: P_O = Y or N, Y/N = primal solution is optimal/is not optimal<br>
Col. 11: P_STATE = -/suboptimal/superoptimal/optimal; depends on P_F and P_O (primal)<br>
Col. 12: D_STATE = -/suboptimal/superoptimal/optimal; depends on D_F and D_O (dual)<br>
Col. 13: D_F = Y or N, Y/N = dual solution is feasible/infeasible<br>
Col. 14: D_O = Y or N, Y/N = dual solution is optimal/is not optimal<br>
Col. 15-18: The values for decision variables (dual solution)<br>
Col. 19-21: The values for surplus variables (dual solution)<br><br>

Reminder: sort solutions according to Z; analyze how their states change with the increase of Z.

In [45]:
from itertools import combinations
from IPython.display import display, HTML
c = np.array([4, 2, 3])
b = np.array([10, 12, 15, 20])
A = np.array([[1,1,0],
     [0,2,1],
     [3,0,2],
     [1,1,1]])
M = np.hstack((A, np.identity(n=4)))



PRIMAL_DUAL_SOLUTIONS = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
for base in combinations(range(7), r=4):
    c_b = padded_c[list(base)]
    ret = getFinalTableau(list(base),c,c_b,b,A,M)
    if ret is None:
        continue
    top_left = ret[3]
    bottom_left = ret[4]
    top_right = ret[1]
    bottom_right = ret[0]
    rs_bottom = ret[2]
    rs_top = c_b@ret[0]@b

    PRIMAL_DUAL_SOLUTIONS[0].append(rs_top)  # Z
    for i,x in enumerate(base):
        PRIMAL_DUAL_SOLUTIONS[1+x].append(rs_bottom[i])
    for x in range(7):
        if x not in base:
            PRIMAL_DUAL_SOLUTIONS[1+x].append(0)
    feasible = False
    if np.all(rs_bottom >= -0.0001):
        feasible = True
        PRIMAL_DUAL_SOLUTIONS[8].append("Y") # Primal feasibility
    else:
        feasible = False
        PRIMAL_DUAL_SOLUTIONS[8].append("N") # Primal feasibility

    optimal = False
    if np.all(np.hstack((top_left,top_right)) >= -0.00001):
        optimal = True
        PRIMAL_DUAL_SOLUTIONS[9].append("Y") # Primal optimality
    else:
        optimal = False
        PRIMAL_DUAL_SOLUTIONS[9].append("N") # Primal optimality

    if optimal and feasible:
        PRIMAL_DUAL_SOLUTIONS[10].append("optimal") # Primal STATE
    elif optimal and not feasible:
        PRIMAL_DUAL_SOLUTIONS[10].append("superoptimal") # Primal STATE
    elif not optimal and feasible:
        PRIMAL_DUAL_SOLUTIONS[10].append("suboptimal") # Primal STATE
    else:
        PRIMAL_DUAL_SOLUTIONS[10].append("-") # Primal STATE
    
    if optimal and feasible:
        PRIMAL_DUAL_SOLUTIONS[11].append("optimal") # Dual STATE
        PRIMAL_DUAL_SOLUTIONS[12].append("Y") # Dual Feasibilty
        PRIMAL_DUAL_SOLUTIONS[13].append("Y") # Dual optimality
    elif optimal and not feasible:
        PRIMAL_DUAL_SOLUTIONS[11].append("suboptimal") # Dual STATE
        PRIMAL_DUAL_SOLUTIONS[12].append("Y") # Dual Feasibilty
        PRIMAL_DUAL_SOLUTIONS[13].append("N") # Dual optimality
    elif not optimal and feasible:
        PRIMAL_DUAL_SOLUTIONS[11].append("superoptimal") # Dual STATE
        PRIMAL_DUAL_SOLUTIONS[12].append("N") # Dual Feasibilty
        PRIMAL_DUAL_SOLUTIONS[13].append("Y") # Dual optimality
    else:
        PRIMAL_DUAL_SOLUTIONS[11].append("-") # Dual STATE
        PRIMAL_DUAL_SOLUTIONS[12].append("N") # Dual Feasibilty
        PRIMAL_DUAL_SOLUTIONS[13].append("N") # Dual optimality

    for i in range(4):
        PRIMAL_DUAL_SOLUTIONS[14+i].append(top_right[i])
    for i in range(3):
        PRIMAL_DUAL_SOLUTIONS[18+i].append(top_left[i])



    
## DISPLAY STORED PAIRS OF SOLUTIONS
df = pd.DataFrame(PRIMAL_DUAL_SOLUTIONS).T
df.columns = ["Z", "x1", "x2", "x3", "slack1", "slack2", "slack3", "slack4", "P_F", "P_O", "P_STATE", "D_STATE", "D_F", "F_O","y1", "y2", "y3", "y4", "sur1", "sur2", "sur3"]
df = df.sort_values(by="Z").reset_index(drop=True)
display(HTML(df.to_html()))

,Z,x1,x2,x3,slack1,slack2,slack3,slack4,P_F,P_O,P_STATE,D_STATE,D_F,F_O,y1,y2,y3,y4,sur1,sur2,sur3
0,-4.0,0,10.0,-8.0,0,0,31.0,18.0,N,N,-,-,N,N,-4.0,3.0,0.0,0.0,-8.0,0.0,0.0
1,0.0,0,0,0,10.0,12.0,15.0,20.0,Y,N,suboptimal,superoptimal,N,Y,0.0,0.0,0.0,0.0,-4.0,-2.0,-3.0
2,12.0,0,6.0,0,4.0,0,15.0,14.0,Y,N,suboptimal,superoptimal,N,Y,0.0,1.0,0.0,0.0,-4.0,0.0,-2.0
3,17.5,10.0,0,-7.5,0,19.5,0,17.5,N,N,-,-,N,N,-0.5,0.0,1.5,0.0,0.0,-2.5,0.0
4,20.0,0,10.0,0,0,-8.0,15.0,10.0,N,N,-,-,N,N,2.0,0.0,0.0,0.0,-2.0,0.0,-3.0
5,20.0,5.0,0,0,5.0,12.0,0,15.0,Y,N,suboptimal,superoptimal,N,Y,0.0,0.0,1.333333,0.0,0.0,-2.0,-0.333333
6,22.5,0,0,7.5,10.0,4.5,0,12.5,Y,N,suboptimal,superoptimal,N,Y,0.0,0.0,1.5,0.0,0.5,-2.0,0.0
7,24.0,-3.0,0,12.0,13.0,0,0,11.0,N,N,-,-,N,N,0.0,0.333333,1.333333,0.0,0.0,-1.333333,0.0
8,27.0,0,2.25,7.5,7.75,0,0,10.25,Y,N,suboptimal,superoptimal,N,Y,0.0,1.0,1.0,0.0,-1.0,0.0,0.0
9,28.0,4.0,6.0,0,0,0,3.0,10.0,Y,N,suboptimal,superoptimal,N,Y,4.0,-1.0,0.0,0.0,0.0,0.0,-4.0
